In [60]:
# pip install plotly
# pip install -U imbalanced-learn

# importing modules
import pandas as pd
import numpy as np
import time
# for downloadind zip files
from io import BytesIO
from zipfile import ZipFile
from urllib.request import urlopen
#for converting .arff format to DataFrame
from scipy.io.arff import loadarff 
#preprocessing
from sklearn import decomposition
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
#plotting the data
import plotly.express as px
#model selection
from sklearn.model_selection import GridSearchCV, RepeatedStratifiedKFold, StratifiedKFold, cross_validate
from sklearn.pipeline import Pipeline
#ML models
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier
from imblearn.ensemble import BalancedBaggingClassifier
#models evaluation
from sklearn.metrics import accuracy_score, balanced_accuracy_score, f1_score
from sklearn.dummy import DummyClassifier

## The Story

You are working in the economic research department of a non-profit organization (a subsidiary of an international bank). The main purpose of your company's work is to prepare an economic overview of the development of central European countries. One of your team's tasks is economic analysis of the public and private sector in Poland. The task is to forecast the possible number of corporate bankruptcies in the country in future years, because colleagues from another team found that bankruptcies in the corporate sector have a significant impact on the level of GDP of the country, which is the main parameter to describe the country's economy. Your task is to build a model that can predict (classify) the bankruptcy of companies in Poland based on financial ratios from accounting statements.

The dataset is about bankruptcy prediction of Polish companies. The data was collected from Emerging Markets Information Service, which is a database containing information on emerging markets around the world. The bankrupt companies were analyzed in the period 2000-2012, while the still operating companies were evaluated from 2007 to 2013.
Basing on the collected data five classification cases were distinguished, that depends on the forecasting period. For this experiment you will use the data from the 4th year, which contains financial rates from 4th year of the forecasting period and corresponding class label that indicates bankruptcy status after 2 years. The data contains 9792 instances (financial statements), 515 represents bankrupted companies, 9277 firms that did not bankrupt in the forecasting period. 

There are 64 features/ attributes (financial rates) in the data:
* Attr1 net profit / total assets, 
* Attr2 total liabilities / total assets
* Attr3 working capital / total assets
* Attr4 current assets / short-term liabilities
* Attr5 [(cash + short-term securities + receivables - short-term liabilities) / (operating expenses - depreciation)] * 365
* Attr6 retained earnings / total assets
* Attr7 EBIT / total assets
* Attr8 book value of equity / total liabilities
* Attr9 sales / total assets
* Attr10 equity / total assets
* Attr11 (gross profit + extraordinary items + financial expenses) / total assets
* Attr12 gross profit / short-term liabilities
* Attr13 (gross profit + depreciation) / sales
* Attr14 (gross profit + interest) / total assets
* Attr15 (total liabilities * 365) / (gross profit + depreciation)
* Attr16 (gross profit + depreciation) / total liabilities
* Attr17 total assets / total liabilities
* Attr18 gross profit / total assets
* Attr19 gross profit / sales
* Attr20 (inventory * 365) / sales
* Attr21 sales (n) / sales (n-1)
* Attr22 profit on operating activities / total assets
* Attr23 net profit / sales
* Attr24 gross profit (in 3 years) / total assets
* Attr25 (equity - share capital) / total assets
* Attr26 (net profit + depreciation) / total liabilities
* Attr27 profit on operating activities / financial expenses
* Attr28 working capital / fixed assets
* Attr29 logarithm of total assets
* Attr30 (total liabilities - cash) / sales
* Attr31 (gross profit + interest) / sales
* Attr32 (current liabilities * 365) / cost of products sold
* Attr33 operating expenses / short-term liabilities
* Attr34 operating expenses / total liabilities
* Attr35 profit on sales / total assets
* Attr36 total sales / total assets
* Attr37 (current assets - inventories) / long-term liabilities
* Attr38 constant capital / total assets
* Attr39 profit on sales / sales
* Attr40 (current assets - inventory - receivables) / short-term liabilities
* Attr41 total liabilities / ((profit on operating activities + depreciation) * (12/365))
* Attr42 profit on operating activities / sales
* Attr43 rotation receivables + inventory turnover in days
* Attr44 (receivables * 365) / sales
* Attr45 net profit / inventory
* Attr46 (current assets - inventory) / short-term liabilities
* Attr47 (inventory * 365) / cost of products sold
* Attr48 EBITDA (profit on operating activities - depreciation) / total assets
* Attr49 EBITDA (profit on operating activities - depreciation) / sales
* Attr50 current assets / total liabilities
* Attr51 short-term liabilities / total assets
* Attr52 (short-term liabilities * 365) / cost of products sold)
* Attr53 equity / fixed assets
* Attr54 constant capital / fixed assets
* Attr55 working capital
* Attr56 (sales - cost of products sold) / sales
* Attr57 (current assets - inventory - short-term liabilities) / (sales - gross profit - depreciation)
* Attr58 total costs /total sales
* Attr59 long-term liabilities / equity
* Attr60 sales / inventory
* Attr61 sales / receivables
* Attr62 (short-term liabilities *365) / sales
* Attr63 sales / short-term liabilities
* Attr64 sales / fixed assets

Also there is a column with class labels:
* 0 - firms that did not bankrupt in the forecasting period
* 1 - represents bankrupted companies

## Data retrieving

Source of the dataset: https://archive.ics.uci.edu/ml/datasets/Polish+companies+bankruptcy+data. Creator: Sebastian Tomczak, Department of Operations Research, Wroclaw University of Science and Technology, Wroclaw, Poland. Donor: Sebastian Tomczak, Maciej Zieba, Jakub M. Tomczak

The dataset files are archived, so we will use ZipFile for reading

In [61]:
# the code for open .zip files from the web is from StackOverFlow: 
# https://stackoverflow.com/questions/5710867/downloading-and-unzipping-a-zip-file-without-writing-to-disk

resp = urlopen('https://archive.ics.uci.edu/ml/machine-learning-databases/00365/data.zip')
myzip = ZipFile(BytesIO(resp.read()))
print(myzip.namelist())

['1year.arff', '2year.arff', '3year.arff', '4year.arff', '5year.arff']


We see 5 files with the datasets in the archive. We will work with the file '4year.arff'

In [62]:
#extracting the file from the zip archive
myzip.extract(member='4year.arff')

'c:\\Users\\lenovo\\python\\FH Kiel DS\\MachineLearning-Exam\\part 2\\4year.arff'

Converting from .arff file format and creating a dataframe with the dataset:

In [63]:
# the code for converting .arff file to DataFrame is from StackOverFlow: 
# https://stackoverflow.com/questions/46401209/how-to-convert-the-arff-object-loaded-from-a-arff-file-into-a-dataframe-format

raw_data = loadarff('4year.arff')
df = pd.DataFrame(raw_data[0])
df.head(5)
print(df.shape)
df.head(4)

(9792, 65)


,Attr1,Attr2,Attr3,Attr4,Attr5,Attr6,Attr7,Attr8,Attr9,Attr10,...,Attr56,Attr57,Attr58,Attr59,Attr60,Attr61,Attr62,Attr63,Attr64,class
0,0.159290,0.46240,0.07773,1.1683,-44.853,0.467020,0.189480,0.82895,1.1223,0.38330,...,0.108990,0.41557,0.89101,0.001422,7.7928,4.9914,119.810,3.0465,3.0560,b'0'
1,-0.127430,0.46243,0.26917,1.7517,7.597,0.000925,-0.127430,1.16250,1.2944,0.53757,...,-0.089372,-0.23704,1.06250,0.150410,5.4327,3.4629,100.970,3.6150,3.4725,b'0'
2,0.070488,0.23570,0.52781,3.2393,125.680,0.163670,0.086895,2.87180,1.0574,0.67689,...,0.054286,0.10413,0.94571,0.000000,7.1070,3.3808,76.076,4.7978,4.7818,b'0'
3,0.136760,0.40538,0.31543,1.8705,19.115,0.504970,0.136760,1.45390,1.1144,0.58938,...,0.102630,0.23203,0.89737,0.073024,6.1384,4.2241,88.299,4.1337,4.6484,b'0'


The dataset have 9792 rows (instnaces) and 65 columns (64 features and 1 class column). All values are numerical

## Data cleaning

The process of data cleaning will consist of the following steps: 
* Analyzing and finding redundant (fully duplicate) features, removing one of the identical features. 
* Correlation analysis of features and making a decision about applying/not applying feature selection. 
* Dealing with missing data

In [64]:
df=df.rename(columns={'class':'Klass'}) #changing the name of the column
df['Klass']  = df['Klass'].replace([b'0', b'1'], [0, 1]) #changing the values of the classes
print(f'Class distribution:\n{df["Klass"].value_counts()}'),
print(f'Total number of missing values: {df.isna().sum().sum()}')

Class distribution:
0    9277
1     515
Name: Klass, dtype: int64
Total number of missing values: 8776


Companies that went bankrupt have class label '0', companies that have maintained their solvency have class label '1'

A first look at the datset revealed two problems:
* A lot of missing values: 8776 rows of total 9792 rows have missing values
* Strongly imbalanced class distribution: 9277 bankrupts and only 515 not bankrupts

Let's analyze the distribution of missing data by columns, highlighting the columns with the largest number of gaps

In [65]:
missing_values = pd.DataFrame(df.isna().sum(), columns=['Number_of_missing_values'])
missing_values = missing_values[missing_values['Number_of_missing_values'] != 0]
missing_values.sort_values(by='Number_of_missing_values', inplace=True,)
missing_values.tail(5)

,Number_of_missing_values
Attr64,231
Attr45,613
Attr60,614
Attr27,641
Attr37,4442


There are some redundunt features in the dataset. To find out we can use domain expertise from accounting and financial analysis, also we can compute correlation between them

In [66]:
# to find out correlation betwween features I used code from StackOverFlow
# https://stackoverflow.com/questions/17778394/list-highest-correlation-pairs-from-a-large-correlation-matrix-in-pandas

def get_redundant_pairs(df):
    '''Get diagonal and lower triangular pairs of correlation matrix'''
    pairs_to_drop = set()
    cols = df.columns
    for i in range(0, df.shape[1]):
        for j in range(0, i+1):
            pairs_to_drop.add((cols[i], cols[j]))
    return pairs_to_drop

def get_top_abs_correlations(df, n=5):
    au_corr = df.corr().abs().unstack()
    labels_to_drop = get_redundant_pairs(df)
    au_corr = au_corr.drop(labels=labels_to_drop).sort_values(ascending=False)
    return au_corr[0:n]

print("Top Absolute Correlations")
print(get_top_abs_correlations(df, 15))

Top Absolute Correlations
Attr8   Attr17    0.999999
Attr7   Attr14    0.999997
Attr4   Attr46    0.999948
Attr43  Attr44    0.999934
Attr39  Attr56    0.999295
Attr2   Attr10    0.999293
Attr28  Attr54    0.998728
Attr3   Attr51    0.997150
Attr10  Attr38    0.996884
Attr2   Attr38    0.996160
Attr16  Attr26    0.994988
Attr19  Attr23    0.994737
Attr38  Attr51    0.994708
Attr4   Attr50    0.994583
Attr46  Attr50    0.994559
dtype: float64


There are a lot of features with strong correlation between them

Features with high correlation are more linearly dependent and hence have almost the same effect on the dependent variable. So, when two features have high correlation, we can drop one of the two features. Not removing feature may decrease efficiency and unnecessary cost. 

But first of all, using domain knowledge we can analize and find redundant (fully duplicate) features, which we can remove without loss of information value:

* Attributes (Attr17) total assets / total liabilities and (Attr8) book value of equity / total liabilities are the same, just different terms are used (total assets = value of equity), and correlation is very strong 0.999999. So we can drop one of them
* Attributes (Attr7) EBIT / total assets and (Attr14) (gross profit + interest) / total assets are the same: EBIT = gross profit + interest. Also correlation is strong 0.999997. So we can drop one of them
* Attributes (Attr4) current assets / short-term liabilities and (Attr46) (current assets - inventory) / short-term liabilities. The financial indicators are almost the same, correlation is strong 0.999948. So we can drop one of them
* Attributes (Attr43) rotation receivables + inventory turnover in days and (Attr44) (receivables * 365) / sales are both explain turnover of the current assets, correlation is strong 0.999934. So we can drop one of them
* Attributes (Attr39) profit on sales / sales and (Attr56) (sales - cost of products sold) / sales are identical (profit on sales = sales - costs of product sold), correlation is strong 0.999295. So we can drop one of them
* Attributes (Attr2) total liabilities / total assets and (Attr10) equity / total assets are are the same, just different terms are used (total liabilities = equity), and correlation is very strong 0.999293. So we can drop one of them
* Attributes (Attr28) working capital / fixed assets and (Attr54) constant capital / fixed assets are the same (working capital = constant capital). correlation is very strong 0.998728. So we can drop one of them

Previously we saw that the column with attribute 37 ((current assets - inventories) / long-term liabilities) is missing 4442 values, almost half of the data in the column. Due to the fact that the column with attribute 37 missed 48% of the values, the column Attr37 could be fully dropped. 
As a rule of thumb if data is missing for more than 60% of the observations, it may be wise to discard it if the variable is insignificant. In our case, less than 60% is missed, which is 48%. However, I think that it would be more appropriate to remove the whole column, as the information will not be a significant loss of information due to the high correlation of the 'Attr37' with other features

In [67]:
df = df.drop(columns=['Attr37', 'Attr8', 'Attr14', 'Attr46', 'Attr43', 'Attr56', 'Attr10', 'Attr54'])
print(df.shape),
print(f'Total number of missing values is: {df.isna().sum().sum()}')

(9792, 57)
Total number of missing values is: 3997


Allocate rows with missing data into a separate table for further analysis and decision-making about deletion or imputation

In [68]:
missing_rows = df[df.isnull().any(axis=1)] #extracting only rows with at leasat one missing value
print(missing_rows.shape)
print(missing_rows['Klass'].value_counts())

(1701, 57)
0    1414
1     287
Name: Klass, dtype: int64


We see that 1,414 lines with missing data (at least one of the 64 values in the line is missing) are in the class "0" (companies that did not become bankrupt), and 287 lines with missing at least one value with a class "1" (companies that became bankrupt)

There are two primary methods to solve the problem of missing data: rows removing or imputation.
* Rows removing. If the data in the unbalanced dataset is missing in the rows belonging to a major class, then it is possible to delete those rows
* Imputation. There are multiple solutions to impute the value of missing data. One of the most common methods of imputing values when dealing with missing data is to calculate the mean or median of the existing observations and to impute new values into the missing places

Allocate the rows with missing data into separate tables depending on the minor class '1'

In [69]:
#dividing missing values from class 1
df_1 = missing_rows[missing_rows['Klass'] == 1]
print(df_1.shape)

(287, 57)


For minor class "1" we can apply the imputation. For this SimpleImputer will be used. It is an univariate imputer for completing missing values with simple strategies of replacing missing values using a descriptive statistic (e.g. mean, median, or most frequent) along each column, or using a constant value. If strategy = “median”, then replace missing values using the median along each column. In a prediction context, simple imputation usually performs poorly when associated with a weak learner. However, with a powerful learner, it can lead to as good or better performance than complex imputation

In [70]:
imp_mean = SimpleImputer(strategy='median', fill_value=True)
result_of_imputation_1 = imp_mean.fit_transform(df_1)
result_of_imputation_1 = pd.DataFrame(result_of_imputation_1, columns=df.columns)
print(result_of_imputation_1.shape)

(287, 57)


Lines with missing data belonging to the major class "0" we will delete completely. Since the general dataset is highly unbalanced and the deletion of lines of the major class should not lead to loss of information

In [71]:
df = df.dropna(how = 'any', axis=0) #dropping only rows with at leasat one missing value

In [72]:
data = pd.concat([df,result_of_imputation_1], ignore_index=True, axis=0)
data = pd.DataFrame(data)
print(f'Shape of the dataset: {data.shape}')
print(f'Total number of missing values: {df.isna().sum().sum()}')
print(f'Class distribution:\n{data["Klass"].value_counts()}')
print(f'Class distribution:\n{data["Klass"].value_counts(normalize= True).round(2)}')

Shape of the dataset: (8378, 57)
Total number of missing values: 0
Class distribution:
0.0    7863
1.0     515
Name: Klass, dtype: int64
Class distribution:
0.0    0.94
1.0    0.06
Name: Klass, dtype: float64


As a result, we deleted completely duplicate columns, deleted rows with missing data from major class "0", and performed an imputation of minor class "1" data. There are no missing data. 

The data is strongly imbalanced:
* 94% of class '0' - not bancrupt
*  6% of class '1' - a bancrupt
With this issue will be dealed by training models with the addition weighted parameter "balanced" and using the balanced_accuracy and f1_accuracy model evaluation metrics

In [73]:
target = data.Klass
attributes = data.drop(columns=['Klass'])
print(f'Features shape: {attributes.shape}')
print(f'Target shape: {target.shape}')

Features shape: (8378, 56)
Target shape: (8378,)


The dataset after cleaning has 8378 lines and 56 features

## Data Visualization

Plotting explained variance: how much variance PCA is able to explain as you increase the number of components. With a higher explained variance, model is able to capture more variability in the dataset, which could potentially lead to better performance when training a model

For using PCA data should be scaled. There are different instruments for scaling. StandardScaler is very useful if data has a gaussian distribution. Using Normalization (row scaling) or MinMaxScaler (column scaler) is useful when data has varying scales and the algorithm does not make assumptions about the distribution. MinMaxScaler is a good technique when we did not know about the distribution of data or when we know the distribution is not gaussian.

In [74]:
# the code from https://plotly.com/python/pca-visualization/
scaler = MinMaxScaler()
attributes_scaled = scaler.fit_transform(attributes.to_numpy())
attributes_scaled = pd.DataFrame(attributes_scaled, index=attributes.index, columns=attributes.columns)
pca = PCA()
pca.fit(attributes_scaled)
exp_var_cumul = np.cumsum(pca.explained_variance_ratio_)
px.area(
    x=range(1, exp_var_cumul.shape[0] + 1), y=exp_var_cumul, labels={"x": "# Components", "y": "Explained Variance"})

The importance of explained variance is demonstrated in the plot above. It can be seen that 9 components provide 81% of variance

To plot the multidimensinal data into 2-D figure we also apply PCA to the dataset. The disvantage is that you will not be able to explain the model's decision with the original features. We will plot the figure with 2 components PC1 and PC2 with maximum variance

In [75]:
#the code from https://plotly.com/python/pca-visualization/
features = attributes_scaled.columns
pca = PCA()
components = pca.fit_transform(attributes_scaled[features])
labels = {str(i): f"PC {i+1} ({var:.1f}%)" for i, var in enumerate(pca.explained_variance_ratio_ * 100)}
df = data
df['Klass'] = df["Klass"].astype(str)
fig = px.scatter_matrix(components, labels=labels, dimensions=range(2), color=df["Klass"])
fig.update_traces(diagonal_visible=False)
fig.show()

The conclusion that can be made from the analysis of the plotted graphs is that the data of the different classes do not have a clear separation between each other. Since classes difficult to distinguish from the others they are not linearly separable

## Experiment Configuration

In order to save time for experiments, we set the number for all iterations at 3, although it is common practice to set it at 10 

In [76]:
NUM_TRIALS = 3
NUM_INNER_REPEATS = 3
NUM_INNER_SPLITS = 3
NUM_OUTER_SPLITS = 3

Experiments have been carried out of SVC models with the following parameters:
* kernels = {'poly', 'rbf'}
* class_weight='balanced'
* pameters grid for polynominal kernel = {'estimator__C': [0.01,0.1,1,10,50,100], 'estimator__degree': [1,2,3,4,5]}
* parameters grid for rbf kernel = {'estimator__C': [0.01,0.1,1,10,50,100], 'estimator__gamma': [0.001, 0.01, 0.1, 1, 10,100]}

The results were the following:
* For the SVC (rbf): f1_weighted accuracy - 0.897232, balanced mean accuracy - 0.894051, fit time - 704 sec.
* For the SVC (poly): f1_weighted accuracy - 0.912472, balanced mean accuracy - 0.9291, fit time - 460 sec.

The best parameters (estimators) for the SVM (rbf) are: C - 0.01, gamma - 1.
The best parameters (estimators) for the SVM (poly) are: C - 0.01, degree - 5.

The problem with using SVM models is its complexity. Support Vector Machines are powerful tools, but their compute and storage requirements increase rapidly with the number of training vectors. The core of an SVM is a quadratic programming problem (QP), separating support vectors from the rest of the training data. For the training it is depending on number of features and samples: O(n_samples^2 * n_features).

For my experiments it took more than 21 minutes to compute the results for SVM (poly) model and more than 50 minutes to compute the results for SVM (rbf) model. 

I've used a tip on practice use from scikitlearn documentation with changing cache size. The size of the kernel cache has a strong impact on run times for larger problems. If you have enough RAM available, it is recommended to set cache_size to a higher value than the default of 200(MB), such as 500(MB) or 1000(MB). In my case increasing the cache size parameter to 2000 did not result in a significant acceleration. 

So, to save time, I did not include the SVM cross validation experiments with the above mentioned parameters, but only left the parameters with which the SVM (poly) model performed best

Configurations, which were used during experiments. Due to the very long execution time of the experiments, the best parameters of the most time-consuming models were chosen.
* tree_grid = {'estimator__criterion': ['gini', 'entropy'], 'estimator__max_depth': [1,2,4,8,16,32,64,128]}
* poly_grid = {'estimator__C': [0.01,0.1,1,10,50,100], 'estimator__degree': [1,2,3,4]}
* adaboost_grid = {'estimator__n_estimators': [10,50,90]}

In [77]:
tree_grid = {'estimator__criterion': ['gini', 'entropy'], 'estimator__max_depth': [1,2,4,8,16]}
poly_grid = {'estimator__C': [0.01], 'estimator__degree': [5]}
adaboost_grid = {'estimator__n_estimators': [5,10,50]}

Creating a pipeline:
* for scaling MinMaxScaler. Transforms features by scaling each feature to a given range.This estimator scales and translates each feature individually such that it is in the given range on the training set, e.g. between zero and one. The motivation to use this scaling include robustness to very small standard deviations of features and preserving zero entries in sparse data.
* for dimensionality reduction Principal component analysis (PCA). PCA is used to decompose a multivariate dataset in a set of successive orthogonal components that explain a maximum amount of the variance.

In [78]:
def get_pipe(estimator):
    return Pipeline([
        ('scaler', MinMaxScaler()),
        ('pca', decomposition.PCA()),
        ('estimator', estimator)])

Since the dataset is highly imbalanced following metrics were choosen for model evaluation:
* f1 (weighted) score. The F1 score can be interpreted as a harmonic mean of the precision and recall, where an F1 score reaches its best value at 1 and worst score at 0. The relative contribution of precision and recall to the F1 score are equal. The formula for the F1 score is: F1 = 2 * (precision * recall) / (precision + recall). Average 'weighted' used to Calculate metrics for each label, and find their average weighted by support (the number of true instances for each label). F1 is high when both, recall and precision are high
* balanced_accuracy score. The balanced accuracy in binary classification problems to deal with imbalanced datasets. It is defined as the average of recall obtained on each class. The best value is 1 and the worst value is 0 

In [79]:
def nested_cv(estimator, grid, features, targets):
    start = time.time()
    f1 = np.zeros((NUM_TRIALS, NUM_OUTER_SPLITS))
    baccs = np.zeros((NUM_TRIALS, NUM_OUTER_SPLITS))
    accs = np.zeros((NUM_TRIALS, NUM_OUTER_SPLITS))
    fit_times = np.zeros((NUM_TRIALS, NUM_OUTER_SPLITS))
    test_times = np.zeros((NUM_TRIALS, NUM_OUTER_SPLITS))

    for i in range(NUM_TRIALS):
        print("Running Outer CV in iteration: ", i, " at ", time.time()-start)
        pipe = get_pipe(estimator)
        inner_cv = RepeatedStratifiedKFold(n_splits=NUM_INNER_SPLITS, n_repeats=NUM_INNER_REPEATS, random_state=i)
        outer_cv = StratifiedKFold(n_splits=NUM_OUTER_SPLITS, shuffle=True, random_state=i)

        clf = GridSearchCV(estimator=pipe, param_grid=grid, cv=inner_cv, scoring=('f1_weighted'), n_jobs=8, refit='f1')
        cv_result = cross_validate(clf, X=attributes, y=target, cv=outer_cv, scoring=('f1_weighted', 'balanced_accuracy', 'accuracy'), n_jobs=8)

        f1[i] = cv_result['test_f1_weighted']
        baccs[i] = cv_result['test_balanced_accuracy']
        accs[i] = cv_result['test_accuracy']
        fit_times[i] = cv_result['fit_time']
        test_times[i] = cv_result['score_time']

    print('Total time: ', (time.time()-start), 'sec.')
    return f1, baccs, accs, fit_times, test_times


In [80]:
def add_result(results, name, f1, accs, baccs, fit_times, test_times):
    row = {
        'name': name,
        'f1_mean': f1.mean(),
        'f1_std': f1.std(),
        'f1_min': f1.min(),
        'f1_max': f1.max(),
        'bacc_mean': baccs.mean(),
        'bacc_std': baccs.std(),
        'bacc_min': baccs.min(),
        'bacc_max': baccs.max(),
        'acc_mean': accs.mean(),
        'fit_time': fit_times.mean(),
        'test_time': test_times.mean()
    }
    return pd.concat([results, pd.DataFrame([row])], ignore_index=True)

In [81]:
results = pd.DataFrame()

## Training ML algorithms (experiments conducting)

In [82]:
decision_tree = DecisionTreeClassifier(random_state=1)
f1, baccs, accs, fit_times, test_times = nested_cv(decision_tree, tree_grid, attributes, target)
results = add_result(results, 'decision_tree', f1, baccs, accs, fit_times, test_times)

Running Outer CV in iteration:  0  at  0.0
Running Outer CV in iteration:  1  at  12.634354591369629
Running Outer CV in iteration:  2  at  27.491225481033325
Total time:  42.79734921455383 sec.


In [83]:
poly_svm = SVC(kernel = 'poly', random_state=1, class_weight='balanced')
f1, baccs, accs, fit_times, test_times = nested_cv(poly_svm, poly_grid, attributes, target)
results = add_result(results, 'SVM (poly)', f1, baccs, accs, fit_times, test_times)

Running Outer CV in iteration:  0  at  0.0
Running Outer CV in iteration:  1  at  26.473761796951294
Running Outer CV in iteration:  2  at  52.185152769088745
Total time:  78.7400324344635 sec.


In [84]:
adaboost = AdaBoostClassifier(random_state=1)
f1, baccs, accs, fit_times, test_times = nested_cv(adaboost, adaboost_grid, attributes, target)
results = add_result(results, 'AdaBoost', f1, baccs, accs, fit_times, test_times)

Running Outer CV in iteration:  0  at  0.0
Running Outer CV in iteration:  1  at  18.140970468521118
Running Outer CV in iteration:  2  at  35.592432737350464
Total time:  53.19987177848816 sec.


## Evaluating the models

In [85]:
results.sort_values(by='f1_mean', ascending=False)

,name,f1_mean,f1_std,f1_min,f1_max,bacc_mean,bacc_std,bacc_min,bacc_max,acc_mean,fit_time,test_time
2,AdaBoost,0.914309,0.002198,0.910521,0.918169,0.934750,0.001966,0.931948,0.936985,0.533376,17.185263,0.064887
1,SVM (poly),0.912613,0.002356,0.909027,0.916753,0.929100,0.003024,0.922994,0.933405,0.540041,24.484511,1.474269
0,decision_tree,0.911715,0.001740,0.909301,0.914859,0.934272,0.002768,0.930541,0.938754,0.521312,10.818076,0.024116


The best models are for AdaBoost, SVM (poly), Decision Tree 

An AdaBoost ensemble classifier is a meta-estimator that begins by fitting a classifier on the original dataset and then fits additional copies of the classifier on the same dataset but where the weights of incorrectly classified instances are adjusted such that subsequent classifiers focus more on difficult cases.

When doing supervised learning, a simple sanity check consists of comparing one’s estimator against simple rules of thumb. DummyClassifier implements several such simple strategies for classification, one of which is 'constant' dummy classifier. It always predicts a constant label that is provided by the user. This is useful for metrics that evaluate a non-majority class.

DummyClassifier makes predictions that ignore the input features. This classifier serves as a simple baseline to compare against other more complex classifiers. The specific behavior of the baseline is selected with the strategy parameter ('constant').

Let's calculate the baseline for the major class ('0'):

In [86]:
dummy_grid_constant_0 = {'estimator__strategy': ['constant'], 'estimator__constant': [0]}
dummy_clf_constant_0 = DummyClassifier(random_state = 1)
f1, baccs, accs, fit_times, test_times = nested_cv(dummy_clf_constant_0, dummy_grid_constant_0, attributes, target)
results = add_result(results, 'dummy_constant_0', f1, baccs, accs, fit_times, test_times)

Running Outer CV in iteration:  0  at  0.0
Running Outer CV in iteration:  1  at  0.6550111770629883
Running Outer CV in iteration:  2  at  1.3120310306549072
Total time:  1.9860424995422363 sec.


In [87]:
results.sort_values(by='f1_mean', ascending=False)

,name,f1_mean,f1_std,f1_min,f1_max,bacc_mean,bacc_std,bacc_min,bacc_max,acc_mean,fit_time,test_time
2,AdaBoost,0.914309,0.002198,0.910521,0.918169,0.934750,0.001966,0.931948,0.936985,0.533376,17.185263,0.064887
1,SVM (poly),0.912613,0.002356,0.909027,0.916753,0.929100,0.003024,0.922994,0.933405,0.540041,24.484511,1.474269
0,decision_tree,0.911715,0.001740,0.909301,0.914859,0.934272,0.002768,0.930541,0.938754,0.521312,10.818076,0.024116
3,dummy_constant_0,0.908769,0.000233,0.908604,0.909098,0.938530,0.000158,0.938417,0.938754,0.500000,0.497573,0.018770


AdaBoost, SVM (poly), Decision Tree performed better than baseline for class '0' according to the F1 mean. But all the models are worse than the baseline according to the balanced accuracy (the percentage of class '0' in the total dataset). So the models show slightly better results than just the probability of choosing the major class, which occurs most often in the dataset

The learning phase and the subsequent prediction of machine learning algorithms can be affected by the problem of imbalanced data set. The balancing issue corresponds to the difference of the number of samples in the different classes. One way to fight this issue is to use classifier which include inner balancing samplers. BalancedBaggingClassifier can deal with it, each bootstrap sample will be further resampled to achieve the auto sampling strategy. Using the BalancedBaggingClassifier model let's try to improve the result

In [88]:
bbc_grid = {'estimator__sampling_strategy': ['auto']}
bbc_adaboost = BalancedBaggingClassifier(base_estimator=adaboost, replacement=False, random_state=1)
f1, baccs, accs, fit_times, test_times = nested_cv(bbc_adaboost, bbc_grid, attributes, target)
results = add_result(results, 'bbc_adaboost', f1, baccs, accs, fit_times, test_times)

Running Outer CV in iteration:  0  at  0.0
Running Outer CV in iteration:  1  at  27.787742376327515
Running Outer CV in iteration:  2  at  55.51246929168701
Total time:  83.92220330238342 sec.


In [89]:
bbc_svm = BalancedBaggingClassifier(base_estimator=poly_svm, replacement=False, random_state=1)
f1, baccs, accs, fit_times, test_times = nested_cv(bbc_svm,bbc_grid, attributes, target)
results = add_result(results, 'bbc_svm', f1, baccs, accs, fit_times, test_times)

Running Outer CV in iteration:  0  at  0.0
Running Outer CV in iteration:  1  at  8.447717189788818
Running Outer CV in iteration:  2  at  17.423977851867676
Total time:  25.668375968933105 sec.


In [90]:
results.sort_values(by='f1_mean', ascending=False)

,name,f1_mean,f1_std,f1_min,f1_max,bacc_mean,bacc_std,bacc_min,bacc_max,acc_mean,fit_time,test_time
2,AdaBoost,0.914309,0.002198,0.910521,0.918169,0.934750,0.001966,0.931948,0.936985,0.533376,17.185263,0.064887
1,SVM (poly),0.912613,0.002356,0.909027,0.916753,0.929100,0.003024,0.922994,0.933405,0.540041,24.484511,1.474269
0,decision_tree,0.911715,0.001740,0.909301,0.914859,0.934272,0.002768,0.930541,0.938754,0.521312,10.818076,0.024116
5,bbc_svm,0.910991,0.001800,0.908443,0.914696,0.919034,0.002588,0.915145,0.923352,0.568822,6.551500,1.675941
3,dummy_constant_0,0.908769,0.000233,0.908604,0.909098,0.938530,0.000158,0.938417,0.938754,0.500000,0.497573,0.018770
4,bbc_adaboost,0.825001,0.007937,0.811020,0.839520,0.767684,0.011688,0.747135,0.789116,0.710195,26.902818,0.717798


Comparison of different models:

* AdaBoost showed the best F1 mean accuracy result (0.9159), also AdaBoost has the best F1 max result (0.9185)
* SVM (poly) shows slightly worse F1 mean accuracy, but haы the better training time
* Balanced Bagging Classifier with SVM base estimator has the best training time and good F1 mean accuracy (0.9117), slightly below the AdaBoost result
* Low values of the standard deviation (f1_std) indicate that the models are seem to be very stable
* Applying the ensemble classifier from the imblearn library did not result in a significant increase in accuracy, but it did increase the speed of learning
* All the models have results just slightly better than the baseline (probability of choosing the major class, which occurs most often in the dataset)


AdaBoost model has the best mean F1 accuracy. Check the best parameters of the model:

In [91]:
def train_prod_model(estimator, grid):
    pipe = get_pipe(estimator)
    cv = RepeatedStratifiedKFold(n_splits=NUM_INNER_SPLITS, n_repeats=NUM_INNER_REPEATS, random_state=1)
    clf = GridSearchCV(estimator=pipe, param_grid=grid, scoring=('f1_weighted'), refit="f1",cv=cv, n_jobs=8)
    clf.fit(attributes, target)
    return clf

In [92]:
clf = train_prod_model(adaboost, adaboost_grid)
print(clf.best_params_)

{'estimator__n_estimators': 50}


The best F1 accuracy of the model is in the case with 50 estimators. The next step we can do is to fine-tune the AdaBoost model by inceasing number of the n_estimators and see if the accuracy is improved:

In [93]:
adaboost_grid_new = {'estimator__n_estimators': [100,150,200]}
clf = train_prod_model(adaboost, adaboost_grid_new)
adaboost_best_results = pd.DataFrame(clf.cv_results_)[['rank_test_score','param_estimator__n_estimators', 'mean_fit_time', 'mean_test_score']]
adaboost_best_results

,rank_test_score,param_estimator__n_estimators,mean_fit_time,mean_test_score
0,3,100,10.312691,0.915752
1,2,150,14.283764,0.917340
2,1,200,17.461081,0.917818


As the number of estimators increases, the accuracy of the model also increases (according to the F1 parameter). However, the increase is very insignificant, which does not affect the final result much. At the same time the training time of the model increases significantly. In this situation it is up to the end user to decide how many n_estimators to use (how much the training time increase will be important)

## Conclusion

Project was done to analyze the dataset and train the ML models to classify companies according to financial indicators from their financial statements of recognition or not recognition of companies as bankrupt. During the work on the project the following steps were done:
* The data was loaded and cleaned up (removing redundunt/duplicate attributes, dealing with missing data: deleting rows and columns, imputation of data)
* PCA was used to analyze the effect of components on the variability in the data, and a graph was plotted with the data in the 2-d dimension
* Experimental conditions are configured. Training (teaching) of the models has been carried out
* The results of the models are evaluated

The result of the obtained models cannot be considered successful: the accuracy of the best AdaBoost model is better than the random guessing between two classes (50%) but is close to the probability of the major class (94%). Source of the problem in imbalanced data. Further steps to improve the performance of the models could be as follows: 
* Using under-sampling methods. One way to fight this issue is to generate new samples in the classes which are under-represented. 
* Using over-sampling methods (SMOTE method)
* Using combination of over- and under-sampling (SMOTETomek, SMOTEENN)

Sources:
* https://archive.ics.uci.edu/ml/datasets/Polish+companies+bankruptcy+data
* https://stackoverflow.com/questions/5710867/downloading-and-unzipping-a-zip-file-without-writing-to-disk
* https://stackoverflow.com/questions/46401209/how-to-convert-the-arff-object-loaded-from-a-arff-file-into-a-dataframe-format
* https://stackoverflow.com/questions/17778394/list-highest-correlation-pairs-from-a-large-correlation-matrix-in-pandas
* https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html
* https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.html
* https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.html
* https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.html
* https://scikit-learn.org/stable/modules/model_evaluation.html



##This work is my own achievement. If I have used external sources, the positions are marked accordingly##